# Azure OpenAI for big data

The Azure OpenAI service can be used to solve a large number of natural language tasks through prompting the completion API. To make it easier to scale your prompting workflows from a few examples to large datasets of examples, we have integrated the Azure OpenAI service with the distributed machine learning library [SynapseML](https://www.microsoft.com/en-us/research/blog/synapseml-a-simple-multilingual-and-massively-parallel-machine-learning-library/). This integration makes it easy to use the [Apache Spark](https://spark.apache.org/) distributed computing framework to process millions of prompts with the OpenAI service. This tutorial shows how to apply large language models at a distributed scale using Azure OpenAI. 

## Prerequisites

The key prerequisites for this quickstart include a working Azure OpenAI resource, and an Apache Spark cluster with SynapseML installed. We suggest creating a Synapse workspace, but an Azure Databricks, HDInsight, or Spark on Kubernetes, or even a python environment with the `pyspark` package will work. 

1. An Azure OpenAI resource – request access [here](https://customervoice.microsoft.com/Pages/ResponsePage.aspx?id=v4j5cvGGr0GRqy180BHbR7en2Ais5pxKtso_Pz4b1_xUOFA5Qk1UWDRBMjg0WFhPMkIzTzhKQ1dWNyQlQCN0PWcu) before [creating a resource](https://docs.microsoft.com/en-us/azure/cognitive-services/openai/how-to/create-resource?pivots=web-portal#create-a-resource)
1. [Create a Synapse workspace](https://docs.microsoft.com/en-us/azure/synapse-analytics/get-started-create-workspace)
1. [Create a serverless Apache Spark pool](https://docs.microsoft.com/en-us/azure/synapse-analytics/get-started-analyze-spark#create-a-serverless-apache-spark-pool)


## Import this guide as a notebook

The next step is to add this code into your Spark cluster. You can either create a notebook in your Spark platform and copy the code into this notebook to run the demo. Or download the notebook and import it into Synapse Analytics

-	[Download this demo as a notebook](https://github.com/microsoft/SynapseML/blob/master/docs/Explore%20Algorithms/OpenAI/OpenAI.ipynb) (select **Raw**, then save the file)
-	Import the notebook. 
    * If you are using Synapse Analytics [into the Synapse Workspace](https://docs.microsoft.com/en-us/azure/synapse-analytics/spark/apache-spark-development-using-notebooks#create-a-notebook) 
    * If your are using Databricks [import into the Databricks Workspace](https://docs.microsoft.com/en-us/azure/databricks/notebooks/notebooks-manage#create-a-notebook). 
    * If you are using Fabric [import into the Fabric Workspace](https://learn.microsoft.com/en-us/fabric/data-engineering/how-to-use-notebook)
-   Install SynapseML on your cluster. See the installation instructions for Synapse at the bottom of [the SynapseML website](https://microsoft.github.io/SynapseML/). 
    * If you are using Fabric, please check [Installation Guide](https://learn.microsoft.com/en-us/fabric/data-science/install-synapseml). This requires pasting an extra cell at the top of the notebook you imported. 
-  	Connect your notebook to a cluster and follow along, editing and running the cells.

## Fill in service information

Next, edit the cell in the notebook to point to your service. In particular set the `service_name`, `deployment_name`, `location`, and `key` variables to match them to your OpenAI service:

In [ ]:
from synapse.ml.core.platform import find_secret

# Fill in the following lines with your service information
# Learn more about selecting which embedding model to choose: https://openai.com/blog/new-and-improved-embedding-model
service_name = "synapseml-openai-2"
deployment_name = "gpt-4.1-mini"
deployment_name_embeddings = "text-embedding-ada-002"

key = find_secret(
    secret_name="openai-api-key-2", keyvault="mmlspark-build-keys"
)  # please replace this line with your key as a string

assert key is not None and service_name is not None

## Create a dataset of prompts

Next, create a dataframe consisting of a series of rows, with one prompt per row. 

You can also load data directly from ADLS or other databases. For more information on loading and preparing Spark dataframes, see the [Apache Spark data loading guide](https://spark.apache.org/docs/latest/sql-data-sources.html).

In [ ]:
# spark session is assumed to be created in the environment already such as in Fabric notebooks with Spark environment
df = spark.createDataFrame(
    [
        ("Hello my name is",),
        ("The best code is code thats",),
        ("SynapseML is ",),
    ]
).toDF("prompt")

## More Usage Examples

### Generating Text Embeddings

In addition to completing text, we can also embed text for use in downstream algorithms or vector retrieval architectures. Creating embeddings allows you to search and retrieve documents from large collections and can be used when prompt engineering isn't sufficient for the task.

For more information on using `OpenAIEmbedding` see our [embedding guide](./Quickstart%20-%20OpenAI%20Embedding).

In [ ]:
from synapse.ml.services.openai import OpenAIEmbedding

embedding = (
    OpenAIEmbedding()
    .setSubscriptionKey(key)
    .setDeploymentName(deployment_name_embeddings)
    .setCustomServiceName(service_name)
    .setTextCol("prompt")
    .setErrorCol("error")
    .setOutputCol("embeddings")
)

display(embedding.transform(df).show(truncate=False))

### Chat Completion

Models such as ChatGPT and GPT-4 are capable of understanding chats instead of single prompts. The `OpenAIChatCompletion` transformer exposes this functionality at scale.

In [ ]:
from synapse.ml.services.openai import OpenAIChatCompletion
from pyspark.sql import Row
from pyspark.sql.types import *


def make_message(role, content):
    return Row(role=role, content=content, name=role)


chat_df = spark.createDataFrame(
    [
        (
            [
                make_message(
                    "system", "You are an AI chatbot with red as your favorite color"
                ),
                make_message("user", "Whats your favorite color"),
            ],
        ),
        (
            [
                make_message("system", "You are very excited"),
                make_message("user", "How are you today"),
            ],
        ),
    ]
).toDF("messages")


chat_completion = (
    OpenAIChatCompletion()
    .setSubscriptionKey(key)
    .setDeploymentName(deployment_name)
    .setCustomServiceName(service_name)
    .setMessagesCol("messages")
    .setOutputCol("chat_completions")
    .setErrorCol("chat_completions_error")
)

display(
    chat_completion.transform(chat_df)
    .select("messages", "chat_completions.choices.message.content")
    .show(truncate=False)
)

### Chat Completion - Advanced Parameters for Reproducible Outputs

SynapseML now supports additional parameters for enhanced control over OpenAI model behavior for reproducible outputs:

- **`temperature`**: Reduces randomness. OpenAI models accept float temperature value between [0, 2]. Set to 0 for best reproducibility.
- **`top_p`**: Controls nucleus sampling as an alternative to temperature. OpenAI models accept float top_p value between [0, 1]. Set close to 0 for best reproducibility.
- **`seed`**: Enables deterministic sampling for reproducible results. Set to any constant int value.


These parameters can be set globally using `OpenAIDefaults` or on individual transformer instances.

In [ ]:
from synapse.ml.services.openai import OpenAIChatCompletion
from synapse.ml.services.openai.OpenAIDefaults import OpenAIDefaults

# Set global defaults including new parameters
defaults = OpenAIDefaults()
defaults.set_deployment_name(deployment_name)
defaults.set_subscription_key(key)
defaults.set_URL(f"https://{service_name}.openai.azure.com/")
defaults.set_temperature(0)
defaults.set_top_p(0.1)
defaults.set_seed(42)

chat_completion = (
    OpenAIChatCompletion()
    .setMessagesCol("messages")
    .setOutputCol("chat_completions")
    .setErrorCol("chat_completions_error")
)

display(
    chat_completion.transform(chat_df)
    .select("messages", "chat_completions.choices.message.content")
    .show(truncate=False)
)

### Structured JSON output with json_schema

Use the new `responseFormat` option `json_schema` to force the model to return JSON strictly matching a provided JSON Schema. This improves reliability for downstream parsing and analytics.

**Key points:**
- Set `responseFormat` to a dict containing `type="json_schema"` and a nested `json_schema` object (JSON string form not supported).
- Provide the schema as a Python dict / Scala Map plus a `name` and optional `strict` flag.
- Bare `json_schema` string is rejected; supply the full dict form.
- Allowed simple String values for `responseFormat`: `text`, `json_object`.
- For `json_schema` minimal validation only checks the presence of the `json_schema` key; the nested schema is passed through unchanged.
- The model output will be constrained to the schema (no extra properties when `additionalProperties: false`).

**Summary table**

| Type        | How to set             | Requires nested schema | Notes |
|-------------|------------------------|------------------------|-------|
| text        | String("text")         | No                     | Raw string output |
| json_object | String("json_object")  | No                     | Model attempts well‑formed JSON (not strictly validated) |
| json_schema | Dict/Map only          | Yes                    | Strict; reject bare string or JSON string form |

Below we request a single field `answer` as structured JSON.

In [ ]:
from synapse.ml.services.openai import OpenAIChatCompletion
from pyspark.sql import Row

# Define the JSON Schema we want the model to satisfy
schema = {
    "type": "object",
    "properties": {"answer": {"type": "string"}},
    "required": ["answer"],
    "additionalProperties": False,
}

# Single user message requesting structured output
messages_df = spark.createDataFrame(
    [
        (
            [
                Row(
                    role="user",
                    content="What is the capital of France?",
                )
            ],
        )
    ]
).toDF("messages")

chat_structured = (
    OpenAIChatCompletion()
    .setSubscriptionKey(key)
    .setDeploymentName(deployment_name)
    .setCustomServiceName(service_name)
    .setMessagesCol("messages")
    .setResponseFormat(
        {
            "type": "json_schema",
            "json_schema": {
                "name": "answer_schema",
                "strict": True,
                "schema": schema,
            },
        }
    )
    .setOutputCol("chat_structured")
    .setErrorCol("chat_structured_error")
)

display(
    chat_structured.transform(messages_df)
    .select("chat_structured.choices.message.content")
    .show(truncate=False)
)
# The returned content should be a JSON object: {\"answer\": \"Paris\"}

### ResponseFormat Options Quick Tests

Below we validate `text`, `json_object`, and `json_schema` across APIs in separate cells for clarity.

In [ ]:
from synapse.ml.services.openai import OpenAIChatCompletion
from pyspark.sql import Row

messages_df = spark.createDataFrame(
    [([Row(role="user", content="Say hello")],)], ["messages"]
)
chat_text = (
    OpenAIChatCompletion()
    .setSubscriptionKey(key)
    .setDeploymentName(deployment_name)
    .setCustomServiceName(service_name)
    .setMessagesCol("messages")
    .setResponseFormat("text")
    .setOutputCol("chat_text")
)
display(
    chat_text.transform(messages_df)
    .select("chat_text.choices.message.content")
    .show(truncate=False)
)

In [ ]:
messages_df = spark.createDataFrame(
    [([Row(role="user", content="Return a JSON object with key greeting")],)],
    ["messages"],
)
chat_json_obj = (
    OpenAIChatCompletion()
    .setSubscriptionKey(key)
    .setDeploymentName(deployment_name)
    .setCustomServiceName(service_name)
    .setMessagesCol("messages")
    .setResponseFormat("json_object")
    .setOutputCol("chat_json_obj")
)
display(
    chat_json_obj.transform(messages_df)
    .select("chat_json_obj.choices.message.content")
    .show(truncate=False)
)

### Responses API Structured JSON with json_schema (flattened form)

For the Responses API, you can pass a flattened `json_schema` dict: top-level `name`, `strict`, and `schema` keys.

In [ ]:
from synapse.ml.services.openai import OpenAIPrompt

schema = {
    "type": "object",
    "properties": {"answer": {"type": "string"}},
    "required": ["answer"],
    "additionalProperties": False,
}
df = spark.createDataFrame(
    [("France", "capital"), ("Germany", "capital")], ["text", "category"]
)
prompt_flat = (
    OpenAIPrompt()
    .setSubscriptionKey(key)
    .setDeploymentName(deployment_name)
    .setCustomServiceName(service_name)
    .setApiType("responses")
    .setApiVersion("2025-04-01-preview")
    .setPromptTemplate("What is the {category} of {text}.")
    .setResponseFormat(
        {
            "type": "json_schema",
            "name": "answer_schema",
            "strict": True,
            "schema": schema,
        }
    )
    .setOutputCol("out_flat")
)
display(prompt_flat.transform(df).select("out_flat").show(truncate=False))

### Invalid: Bare json_schema String

Attempting to set `responseFormat` to the bare string `json_schema` should raise an error.

In [ ]:
try:

    OpenAIPrompt().setResponseFormat("json_schema")

except Exception as e:

    print("Expected error:", e)

## Working with Usage Statistics

The following examples show how to enable `returnUsage` for different OpenAI API surfaces and inspect the usage maps that are now returned alongside the model output.


### Chat Completions with `returnUsage`


In [ ]:
from synapse.ml.services.openai import OpenAIPrompt

chat_usage_prompt = (
    OpenAIPrompt()
    .setSubscriptionKey(key)
    .setDeploymentName(deployment_name)
    .setCustomServiceName(service_name)
    .setApiType("chat_completions")
    .setReturnUsage(True)
    .setPromptTemplate("Provide a fun fact about {topic}.")
    .setOutputCol("chat_with_usage")
)

chat_usage_df = spark.createDataFrame([("Azure AI",)], ["topic"])
chat_usage_result = chat_usage_prompt.transform(chat_usage_df)
display(
    chat_usage_result.select("chat_with_usage.response", "chat_with_usage.usage").show(
        truncate=False
    )
)

### Responses API with `returnUsage`


In [ ]:
responses_usage_prompt = (
    OpenAIPrompt()
    .setSubscriptionKey(key)
    .setDeploymentName(deployment_name)
    .setCustomServiceName(service_name)
    .setApiType("responses")
    .setApiVersion("2025-04-01-preview")
    .setReturnUsage(True)
    .setPromptTemplate("List two key capabilities of {topic}.")
    .setOutputCol("responses_with_usage")
)

responses_usage_df = spark.createDataFrame([("Azure OpenAI",)], ["topic"])
responses_usage_result = responses_usage_prompt.transform(responses_usage_df)
display(
    responses_usage_result.select(
        "responses_with_usage.response", "responses_with_usage.usage"
    ).show(truncate=False)
)

### Embeddings with `returnUsage`


In [ ]:
from synapse.ml.services.openai import OpenAIEmbedding

embedding_usage = (
    OpenAIEmbedding()
    .setSubscriptionKey(key)
    .setCustomServiceName(service_name)
    .setDeploymentName(deployment_name_embeddings)
    .setReturnUsage(True)
    .setTextCol("text")
    .setOutputCol("embedding_with_usage")
)

embedding_usage_df = spark.createDataFrame(
    [("Usage statistics help monitor token consumption.",)], ["text"]
)
embedding_usage_result = embedding_usage.transform(embedding_usage_df)
display(
    embedding_usage_result.select(
        "embedding_with_usage.response", "embedding_with_usage.usage"
    ).show(truncate=False)
)